### Ingesta del archivo "movie_genre.json"

### Paso 1 - Leer el archivo JSON usando "DataFrameReader" de Spark

In [0]:
dbutils.widgets.text("p_enviroment", "")
v_enviroment = dbutils.widgets.get("p_enviroment")

In [0]:
dbutils.widgets.text("p_file_date", "2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
from pyspark.sql.functions import current_timestamp,lit,col


In [0]:
movie_genre_schema = "movieId INT, genreId INT"

In [0]:
movie_genre_df = spark.read.schema(movie_genre_schema).json(f"{bronze_folder_path}/{v_file_date}/movie_genre.json")

In [0]:
movie_genre_df.printSchema()

In [0]:
display(movie_genre_df)

#### Paso 2 - Cambiar el nombre a las columnas y añadir "ingestion_date" y "enviroment"

In [0]:
movie_genre_final_df = add_ingestion_date(movie_genre_df).withColumnRenamed("movieId","movie_id").withColumnRenamed("genreId","genre_id").withColumns({"enviroment":lit(v_enviroment),"file_date":lit(v_file_date)})

In [0]:
display(movie_genre_final_df)

#### Paso 4 - Escribir la salida en un formato "Parquet"

In [0]:
merge_condition = 'tgt.movie_id = src.movie_id AND tgt.genre_id = src.genre_id AND tgt.file_date = src.file_date'
merge_delta_lake(movie_genre_final_df, 'movie_silver', 'movies_genres', silver_folder_path, merge_condition, 'file_date')

In [0]:
%fs
ls /mnt/historialpeliculas/silver/movies_genres/

In [0]:
%sql
SELECT file_date, COUNT(1)
FROM movie_silver.movies_genres
GROUP BY file_date;

In [0]:
dbutils.notebook.exit("Success")